In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table, vstack
from astropy import units as u

In [2]:
sample = Table.read('Gr8stars_GaiaeDR3_TIC_hires.csv')

In [3]:
sample[0:3]

_RAJ2000,_DEJ2000,EDR3Name,RA_ICRS,e_RA_ICRS,DE_ICRS,e_DE_ICRS,Source,Plx,e_Plx,pmRA,e_pmRA,pmDE,e_pmDE,Nper,RUWE,FG,e_FG,Gmag,e_Gmag,FBP,e_FBP,BPmag,e_BPmag,FRP,e_FRP,RPmag,e_RPmag,EBPRP,BPRP,RVDR2,e_RVDR2,Tefftemp,loggtemp,FeHtemp,PS1,SDSSDR13,SkyMapper2,URAT1,GmagCorr,e_GmagCorr,FGCorr,EBPRPCorr,RAJ2000,e_RAJ2000,DEJ2000,e_DEJ2000,_RAJ2000_1,_DEJ2000_1,TIC,RAJ2000_1,DEJ2000_1,HIP,TYC,UCAC4,2MASS,objID,WISEA,GAIA,APASS,KIC,SG,Ref,r_Pos,pmRA_1,e_pmRA_1,pmDE_1,e_pmDE_1,r_pm,Plx_1,e_Plx_1,r_Plx,GLON,GLAT,ELON,ELAT,Bmag,e_Bmag,u_e_Bmag,Vmag,e_Vmag,u_e_Vmag,umag,e_umag,gmag,e_gmag,rmag,e_rmag,imag,e_imag,zmag,e_zmag,Jmag,e_Jmag,Hmag,e_Hmag,Kmag,e_Kmag,q_2MASS,W1mag,e_W1mag,W2mag,e_W2mag,W3mag,e_W3mag,W4mag,e_W4mag,Gmag_1,e_Gmag_1,Tmag,e_Tmag,f_Tmag,Flag,Teff,s_Teff,logg,s_logg,MH,e_MH,Rad,s_Rad,Mass,s_Mass,rho,s_rho,LClass,Lum,s_Lum,Dist,s_Dist,EBV,s_EBV,Ncont,Rcont,Disp,m_TIC,Prior,e_EBV,E_EBV,f_EBV,e_Mass,E_Mass,e_Rad,E_Rad,e_rho,E_rho,e_logg,E_logg,e_Lum,E_Lum,e_Dist,E_Dist,r_Dist,e_Teff,E_Teff,r_Teff,BPmag_1,e_BPmag_1,RPmag_1,e_RPmag_1,q_Gaia,r_Vmag,r_Bmag,Clist,e_RAJ2000_1,e_DEJ2000_1,RAOdeg,DEOdeg,e_RAOdeg,e_DEOdeg,RadFl,WDFl,ID,Sloan,n_hires_obs_all,n_hires_obs_iodout,hires_snr,hires_maxres,n_hires_obs_maxres_iodout,hires_maxres_snr
float64,float64,str29,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,int64,str12,str10,str16,int64,str19,int64,int64,int64,str4,str7,str7,float64,float64,float64,float64,str6,float64,float64,str5,float64,float64,float64,float64,float64,float64,int64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str19,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str5,str5,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str5,float64,float64,float64,float64,float64,float64,int64,float64,str5,str5,float64,float64,float64,str9,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str6,float64,float64,str6,float64,float64,float64,float64,int64,str8,str8,str13,float64,float64,float64,float64,float64,float64,int64,int64,int64,str5,int64,int64,int64,float64,int64,float64
41.5608721605419,5.5925913340762,Gaia EDR3 6478258087046528,41.56120683549,0.0204,5.59239346316,0.0187,6478258087046528,17.5306,0.0239,74.943,0.025,-44.521,0.023,15,0.965,14901000.0,7644.3,7.754348,0.002811,8553300.0,15579.0,8.008211,0.00342,9250000.0,16715.0,7.332537,0.004258,1.195,0.675674,6.05,0.94,6250.0,4.5,-0.25,-1,1237670015123456040,-1,478010437,7.754348,0.002811,14901000.0,0.004,41.56087215996,0.3943,5.59259133442,0.3768,41.5608716480211,5.5925905296155,318841562,41.56087164802,5.59259052962,12925,0053-01184-1,478-004112,02461462+0535333,-1,J024614.64+053532.7,6478258087046528,38122243,-1,STAR,tmgaia2,tmgaia2,75.059,0.087,-44.335,0.091,gaia2,17.5166,0.0545,gaia2,167.52548808968,-47.09519869347,40.87665792668,-9.9752554377,8.368,0.016,0,7.88,0.03,0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,6.859,0.035,6.632,0.051,6.517,0.036,AAA-111-111-000-0-0,6.466,0.071,6.39,0.025,6.494,0.015,6.383,0.068,7.74137,0.000811,7.377,0.006,gbprp,spect,6143.0,124.8,4.3304,0.078,nan,nan,1.219,0.054,1.16,0.155,0.6402,0.1395,DWARF,1.90692,0.06361,56.9954,0.1783,nan,nan,34,0.0124,--,False,0.004519,nan,nan,--,0.13,0.18,0.053,0.054,0.1116,0.1675,0.0618,0.0941,0.04722,0.08,0.1778,0.1788,bj2018,150.1,99.5,spect,8.03108,0.003429,7.34148,0.003406,1,hipvmag,apassdr9,--,1.34381440126,1.411456771,41.56119636609,5.59239964333,0.04583211778,0.03991938701,1,0,38567162,Sloan,2,0,0,0.0,0,0.0
45.9617289426184,6.13330135022

In [4]:
table = sample['_RAJ2000', '_DEJ2000']
table.rename_column('_RAJ2000', 'ra_j2000')
table.rename_column('_DEJ2000', 'dec_j2000')

In [5]:
# names:
table['primary_name'] = sample['EDR3Name'] # CHANGE THIS LATER
table['gaiadr2_source_id'] = sample['GAIA']
table['gaiaedr3_source_id'] = sample['Source']
table['hip'] = sample['HIP']
table['tic'] = sample['TIC']
table['2mass'] = sample['2MASS']
table['wisea'] = sample['WISEA']

In [6]:
# gaia:
table['parallax'] = sample['Plx']
table['parallax_error'] = sample['e_Plx']
table['pmra'] = sample['pmRA']
table['pmra_error'] = sample['e_pmRA']
table['pmdec'] = sample['pmDE']
table['pmdec_error'] = sample['e_pmDE']
table['gmag'] = sample['Gmag']
table['bp_rp'] = sample['BPRP']
table['radial_velocity'] = sample['RVDR2']
table['radial_velocity_error'] = sample['e_RVDR2']

In [7]:
# spectra:
table['spectrum_hires'] = ''
table['spectrum_hires_resolution'] = sample['hires_maxres']
table['spectrum_hires_snr'] = sample['hires_maxres_snr']
table['spectrum_hires_nobs'] = sample['n_hires_obs_maxres_iodout']

In [8]:
table['spectrum_hires'][sample['n_hires_obs_iodout'] > 0] = 'y'

In [11]:
table[0:10]

ra_j2000,dec_j2000,primary_name,gaiadr2_source_id,gaiaedr3_source_id,hip,tic,2mass,wisea,parallax,parallax_error,pmra,pmra_error,pmdec,pmdec_error,gmag,bp_rp,radial_velocity,radial_velocity_error,spectrum_hires,spectrum_hires_resolution,spectrum_hires_snr,spectrum_hires_nobs
float64,float64,str29,int64,int64,int64,int64,str16,str19,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str1,float64,float64,int64
41.5608721605419,5.5925913340762,Gaia EDR3 6478258087046528,6478258087046528,6478258087046528,12925,318841562,02461462+0535333,J024614.64+053532.7,17.5306,0.0239,74.943,0.025,-44.521,0.023,7.754348,0.675674,6.05,0.94,,0.0,0.0,0
45.9617289426184,6.1333013502295,Gaia EDR3 6963761190199424,6963761190199424,6963761190199424,14258,365294034,03035081+0607598,J030350.97+060800.4,31.9791,0.0285,232.147,0.03,49.838,0.026,6.792566,0.709241,24.68,0.2,y,71700.0,322.4298373289916,5
45.1504065574723,5.9860624232757,Gaia EDR3 6989290475763456,6989290475763456,6989290475763456,14023,347572104,03003613+0559096,--,30.3219,0.0256,698.233,0.028,-158.181,0.023,7.899625,0.993625,70.99,0.27,y,71700.0,53.150729063673246,2
45.0117163658645,7.7497551039991,Gaia EDR3 8479094371605632,8479094371605632,8479094371605632,13976,387609654,03000280+0744590,--,42.0781,0.0278,328.026,0.032,21.181,0.027,7.719903,1.103361,28.64,0.13,y,71700.0,225.22211259110415,2
52.0543127100169,7.3022236665417,Gaia EDR3 9994255754844928,9994255754669184,9994255754844928,16158,399965184,03281303+0718080,J032813.02+071807.1,11.2177,0.032,-14.804,0.037,-82.405,0.03,7.872291,0.744398,-29.51,0.18,,0.0,0.0,0
49.3599428438048,7.6558109779928,Gaia EDR3 10584899657116672,10584899657116672,10584899657116672,15304,445925825,03172638+0739208,J031726.49+073920.5,21.2723,0.021,166.357,0.026,-6.305,0.021,7.265665,0.714235,31.35,0.2,y,71700.0,206.86952409671173,4
49.3865837467127,7.6901546137745,Gaia EDR3 10608573516849536,10608573516849536,10608573516849536,15310,445925813,03173278+0741245,J031732.89+074124.4,21.2261,0.0218,168.807,0.029,-6.387,0.022,7.616955,0.767401,31.9,0.15,y,71700.0,206.86952409671173,4
53.7643841868456,11.045868704137,Gaia EDR3 13193284835783552,13193284835783552,13193284835783552,16715,416720026,03350345+1102450,J033503.45+110245.1,10.2238,0.0448,-12.076,0.048,-6.731,0.035,7.868121,0.707161,6.11,0.26,,0.0,0.0,0
48.6967786961401,8.9807932623103,Gaia EDR3 14163878724940672,14163878724940672,14163878724940672,15099,365441779,03144722+0858509,--,43.2649,0.0252,400.579,0.029,-404.425,0.023,7.586812,1.047251,-16.92,0.19,y,71700.0,129.4488315899375,2


In [12]:
table.write('Gr8stars_GaiaeDR3_formatted.csv', overwrite=True)